# Libraries

In [1]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util

**Load & Preprocess Dataset**

In [4]:

file_path = "/kaggle/input/shl-dataset-3/SHL dataset.xlsx"
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names


data = xls.parse(sheet_names[0])
data

,Relevant Job Roles,"Knowledge, Skills, Abilities",Test,time(min),Test Type,Remote Testing,Adaptive/IRT Support
0,"Accounts Receivable Specialist, Bill and Accou...","Invoice processing, Cheque and cash receipts ,...",Accounts Receivable Simulation,8,simulations,Yes,No
1,"General and Operations Manager, Business Manag...","Capacity planning and Layout planning,\nProces...",Operations Management,10,Knowledge and Skills,Yes,No
2,".Net Developer, Software Developer, Software E...","MVC architecture , MVC life cycle , Inversion ...",.NET MVC,8,Knowledge and Skills,Yes,No
3,"Programmers, Application Developers",Application Development  Application Foundati...,.NET Framework 4.5,30,Knowledge and Skills,Yes,No
4,".Net Developer, Software Developer, Software E...",Model View Communication\n• WPF data bindings\...,.NET MVVM,5,Knowledge and Skills,Yes,No
...,...,...,...,...,...,...,...
83,"Software Developer, Web Developer, Frontend De...","• Control statements, loops and comparison • D...",JavaScript,9,knowledge and skills,Yes,No
84,"PE- Protocol Developer, Datacom Developer, Sys...",• The Linux Platform • Linux Command Line • Li...,Linux Operating System,15,knowledge and skills,Yes,No
85,"Hospitality Suite, The Gaming Manager solution...","Quantitative Ability, Assertiveness, Managemen...",Gaming Manager – One Sitting,55,"simulations, knowledge and skills, Ability and...",Yes,No
86,"Marketing Manager,Market Research Analyst and ...",• The STP process • Marketing strategies and p...,Marketing,9,knowledge and skills,Yes,No


In [5]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  
    return text.strip()

data['Relevant Job Roles'] = data['Relevant Job Roles'].apply(clean_text)
data['Knowledge, Skills, Abilities'] = data['Knowledge, Skills, Abilities'].apply(clean_text)
data['Combined'] = data['Relevant Job Roles'] + " " + data['Knowledge, Skills, Abilities']

**Embed Dataset with Sentence Transformer**

In [6]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
dataset_embeddings = model.encode(data['Combined'].tolist(), convert_to_tensor=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

**Recommend Top-N Tests**

In [7]:
def get_user_input():
    roles = input()
    cleaned_roles = clean_text(roles)
    return cleaned_roles

def recommend_tests(user_input, top_n=10):
    """Generate test recommendations based on user input"""
    user_embedding = model.encode([user_input], convert_to_tensor=True)
    
    cos_scores = util.cos_sim(user_embedding, dataset_embeddings)[0]
    top_results = cos_scores.topk(top_n)
    
  
    recommendations = []
    for score, idx in zip(top_results[0], top_results[1]):
        record = data.iloc[idx.item()]
        recommendations.append({
            'Test': record['Test'],
            # 'Relevance Score': f"{score:.2f}",
            'Time (min)': record['time(min)'],
            'Test Type': record['Test Type'],
            'Remote Testing': record['Remote Testing'],
            'Adaptive/IRT Support': record['Adaptive/IRT Support']
        })
    
    return pd.DataFrame(recommendations)

In [12]:
if __name__ == "__main__":
    user_input = get_user_input()
    recommendations = recommend_tests(user_input)
    print(recommendations.to_string(index=False))

 I am hiring for an analyst and wants applications to screen using Cognitive and personality tests,  what options are available within 45 mins.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                            Test  Time (min)            Test Type Remote Testing Adaptive/IRT Support
Interviewing and Hiring Concepts          35 knowledge and skills            Yes                   No
                    Econometrics          11 knowledge and skills            Yes                   No
                      Data Entry           4 knowledge and skills            Yes                   No
                Basic Statistics          10 Knowledge and Skills            Yes                   No
     Accounts Payable Simulation           8          simulations            yes                   No
                Accounts Payable           9 Knowledge and Skills            Yes                   No
             Accounts Receivable          13 Knowledge and Skills            Yes                   No
  Accounts Receivable Simulation           8          simulations            Yes                   No
             Microsoft Excel 365          25 knowledge and skills            Yes  